# RNC Clasificación De Perros Y Gatos

Este proyecto esta diseñado para crear la estructura basica para predecir imágenes de perros y gatos. Quiza no es un problema a resolver en la vida cotidiana, sin embargo, requiere toda la estructura para solucionar proyectos más complejos.

En este ejercicio tenemos un conjunto de datos que no tiene una estructura rectangular, que es la forma común en archivos .csv. En este caso los datos están conformados por una estructura matricial. 

En este caso aún es necesario tener variables independientes (etiquetas) que nos ayuden a predecir las variables dependientes (imagenes). En este caso no tenmos una variable independiente de una sola columna. Para este proyecto, la información se ha estructurado separando las imagenes en carpetas para train y test. A su vez estas estan separadas en carpetas con solo imagenes de perros y gatos. 

Por lo tanto, es necesario contruir un algoritmo que este en la capacidad de etiquetar o interpretar la categoria de imágen dentro de cada carpeta. La opción mas rapida es usar el nombre de la carpeta como etiqueta de perro o gato. 


In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [2]:
directory = os.getcwd()
directory

'C:\\Users\\josel\\RNC_ClasificarPerrosYGatos'

In [3]:
#Importar librerias y paquetes para RNC
from keras.models import Sequential         #Inicializador de redes neuronales
from keras.layers import Convolution2D, Conv2D     #Crea red convolucional para matriz 2D
from keras.layers import MaxPooling2D       #Reduce la información de la convolución dejando los elementos mas importantes
from keras.layers import Flatten            #Al terminar todas la convoluciones la información va a una RNN que requiere la
                                            #la información en un vector
from keras.layers import Dense              #Crea capas densas en una RNN basica

## Creando el modelo

In [4]:
#Inicializar los pesos de la red neuronal convolucional
classifier = Sequential() #Sequential se encarga de este proceso

**Paramertros Convolution2D**
Crear la capa de convolución, este proceso aplicara diferentes filtros que segmentan la información mas valiosa de la imagen
- **filters:** Número de caracteristicas o filtros que deseo detectar, se recomienda usar valores que sean potencias de dos
- **kernel_size:** Tamaño del filtro (ventana cuadrada) se recomienda valores impares. Obtendremos matrices mas pequeñas a la imagen original, una ventana de 3x3 reducira en una fila y una columna la imagen. Es necesario entregar los dos valores filas y columnas del kernel.
- **strides:** Nos indica el valor del paso, y donde se centra el valor del kernel, debe ser un valor diferente a uno.
- **padding:** Se usa para indicar si usaremos posiciones mas allá del tamaño original de la mátriz para posicionar el recorrido del kernel. 
- **data_format:** Especifica el tamaño de las imagenes, es importante especificarlo en los casos donde la imágen no es cuadrada o tenemos mas de un canal de color. 
- **input_shape:** Se recomienda tener tamaños de imagenes pequeñas y con dimensiones multiplos de potencias de dos. En ocaciones tambien podemos definier el tamaño del lote o batch, no siemprre es necesario hacerlo en este punto. Por ejemplo una entrada con un batch=4, imagen de 28x28 y tres canales de color:  **input_shape = (4, 28, 28, 3)** si no queremos usar el batch tendriamos **input_shape = (28, 28, 3)**.

**NOTA** En tensor flow se estipula primero el tamaño de los canales y despues el tamaño.

-**activation:** Usaremos como activación una función de rectificación unitaria RELU. 

In [5]:
#32 filtros, kernerl de 3x3, imagenes rgb de 64x64, 
classifier.add(Conv2D(32, 3,3, input_shape = (64,64,3), activation = "relu"))

## Max pooling
Este proceso reduce el tamaño de la matriz de caracteristicas que encontramos en el paso anterior, lo reduciremos encontrando un maximo, minimo, o promedio. Esto minimiza la cantidad de datos elimina la información memos importante. Sin embargo es una perdida de información, así que no debemos abusar de este proceso. 
**Parametros**
- pool_size: El tamaño de la ventana por defecto 2x2
- Strides: Pasos de movimiento de la ventana por defecto tiene el mismo tamaño de pool_size

In [6]:
#Poling con ventana de 2x2
classifier.add(MaxPooling2D((2,2)))

## Flattening
Este paso aplana todos los resultados de la ultima etaoa de max pooling para entregar la información a una red neuronal convencional, osea una estructura en vertical, un vector de datos en columna. Este vector tendra en cada una de sus filas el valor de una caracterisitca encontrada en la convolución, de esta manera lo que entregaremos no sera el valor de un pixel, en su lugar encontregaremos información. Esta capa no requiere especificar ningún parametro

In [7]:
#FLattening, vector de caracteristicas de la RNC
classifier.add(Flatten())

## Conectando la información a la rnn de clasificación
Debemos crear una capa oculta completamente conectada a la que le entregaremos la información encontrada en la convolución, está sección sera la encargada de clasificar las imagenes.

No es necesario especificar el tamaño de la entra de la primera capa debido a que esta información estara dada por la capa flattening especificada en el paso anterior, debemos indicar el número de neuronas de la capa oculta, este valor debe ser un multiplo de potencias de 2, generalmente el mas cercano al promedio entre el numero de valores de entrada (32x32) y salida(2). En este caso no usaremos la regla empirica y usaremos 128 y una función de activación RELU.

**NOTA** En versiones pasadas la salida se especificaba como output_dim ahora usamos la palabra units.

In [9]:
classifier.add(Dense(units = 128, activation = "relu"))

### Capa de salida o categorica
En este caso tenemos una clasificación, por lo tanto es necesario usar en la capa final una activación sigmoide, en caso de tener mas de dos categorias es necesario usar softmax y entropia cruzada para que todas las probabilidades queden alineadas a 100%. En este caso solo requiero un nodo de salida el cual tomara un valor de 0 o 1 para clasificar como perros o gatos. 

In [10]:
classifier.add(Dense(units = 1, activation = "sigmoid"))

## Entrenamiento del modelo
https://www.tutorialspoint.com/keras/keras_model_compilation.htm
Argumentos:
- Optimizer: Método para encontrar el valor optimo usaremos adam
- Loss: Función para medir el error entre cada iteración, en problemas de clasificación buscamos funciones relacionadas con la regresión logistica o clasificadores categoricos. En este caso podemos usar una entropia binaria por tener solo dos salidas, en caso contrario debemos usar entropia categorica. 
- Metrics: Es la medida de que tan bueno es nuestro modelo, es similar a la función de perdida pero no se usa en el entrenamiento. Podemos especificar mas de una medida.

In [12]:
classifier.compile(optimizer = "adam", loss = "binary_cross_entropy", metrics = ["accuracy"])